**IMPORT PACKAGES**

In [3]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import os
#VISUALIZATION
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

**READING RAW DATA FROM CSV FILES**

In [24]:
%%time
df_list = []
source='/Users/antoniobravomunoz/Documents/DATA_SCIENCE_MASTER/TFM/Data/Historico/Prueba/'

for file in os.listdir(source):
    df_aux = pd.read_csv(source+file,sep=';')
    df_list.append(df_aux)

df = df.append([df for df in df_list],ignore_index=True)
#df.to_csv('final.csv', index=False)

CPU times: user 46.6 s, sys: 27.6 s, total: 1min 14s
Wall time: 1min 12s


**DATA PREPARATION**

- Formating "Fecha" Column and splitting it on "Date" and "Hora"

In [27]:
%%time
df['fecha']=pd.to_datetime(df['fecha'], format='%Y-%m-%d %H:%M:%S')
df['Fecha']=df["fecha"].dt.date
#Creating new columns spiltting Date and Hour of the day.
df['Hora']=df["fecha"].dt.time
df["Date"]=df["fecha"]
#Drop original column fecha
df = df.drop('fecha', 1)

CPU times: user 1min 1s, sys: 1min 13s, total: 2min 15s
Wall time: 2min 35s


In [28]:
df.dtypes

id                              int64
tipo_elem                      object
intensidad                      int64
ocupacion                       int64
carga                           int64
vmed                            int64
error                          object
periodo_integracion             int64
Fecha                          object
Hora                           object
Date                   datetime64[ns]
dtype: object

- Obtaining X e Y UTM coordinates from others files

Finally, we only keep in mind points measuremente places on the last month registed. By this way, we filer only for the points that they are currently working. These are only the interested ones for us.

In [42]:
sourceMP='/Users/antoniobravomunoz/Documents/DATA_SCIENCE_MASTER/TFM/Data/HistoricoPuntosMedidaUbic/pmed_ubicacion_09_2018.csv'
dfplaces=pd.read_csv(sourceMP,sep=';',encoding='ISO-8859-1',decimal=",")

In [43]:
dfplaces.dtypes

tipo_elem     object
id             int64
cod_cent      object
nombre        object
x            float64
y            float64
dtype: object

- Transforming Coordinates from UTM to Latitude and Longitude.

    Code function source: https://stackoverflow.com/questions/343865/how-to-convert-from-utm-to-latlng-in-python-or-javascript

    Modifications: 
    - Madrid zone value: 30
    - Values returned like a list of float64

In [45]:
def utmToLatLng(easting, northing, northernHemisphere=True,zone=30):
    #Defining default Madrid zone with the value: 30
    if not northernHemisphere:
        northing = 10000000 - northing

    a = 6378137
    e = 0.081819191
    e1sq = 0.006739497
    k0 = 0.9996

    arc = northing / k0
    mu = arc / (a * (1 - math.pow(e, 2) / 4.0 - 3 * math.pow(e, 4) / 64.0 - 5 * math.pow(e, 6) / 256.0))

    ei = (1 - math.pow((1 - e * e), (1 / 2.0))) / (1 + math.pow((1 - e * e), (1 / 2.0)))

    ca = 3 * ei / 2 - 27 * math.pow(ei, 3) / 32.0

    cb = 21 * math.pow(ei, 2) / 16 - 55 * math.pow(ei, 4) / 32
    cc = 151 * math.pow(ei, 3) / 96
    cd = 1097 * math.pow(ei, 4) / 512
    phi1 = mu + ca * math.sin(2 * mu) + cb * math.sin(4 * mu) + cc * math.sin(6 * mu) + cd * math.sin(8 * mu)

    n0 = a / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (1 / 2.0))

    r0 = a * (1 - e * e) / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (3 / 2.0))
    fact1 = n0 * math.tan(phi1) / r0

    _a1 = 500000 - easting
    dd0 = _a1 / (n0 * k0)
    fact2 = dd0 * dd0 / 2

    t0 = math.pow(math.tan(phi1), 2)
    Q0 = e1sq * math.pow(math.cos(phi1), 2)
    fact3 = (5 + 3 * t0 + 10 * Q0 - 4 * Q0 * Q0 - 9 * e1sq) * math.pow(dd0, 4) / 24

    fact4 = (61 + 90 * t0 + 298 * Q0 + 45 * t0 * t0 - 252 * e1sq - 3 * Q0 * Q0) * math.pow(dd0, 6) / 720

    lof1 = _a1 / (n0 * k0)
    lof2 = (1 + 2 * t0 + Q0) * math.pow(dd0, 3) / 6.0
    lof3 = (5 - 2 * Q0 + 28 * t0 - 3 * math.pow(Q0, 2) + 8 * e1sq + 24 * math.pow(t0, 2)) * math.pow(dd0, 5) / 120
    _a2 = (lof1 - lof2 + lof3) / math.cos(phi1)
    _a3 = _a2 * 180 / math.pi

    latitude = 180 * (phi1 - fact1 * (fact2 + fact3 + fact4)) / math.pi

    if not northernHemisphere:
        latitude = -latitude

    longitude = ((zone > 0) and (6 * zone - 183.0) or 3.0) - _a3

    return [latitude, longitude]

In [46]:
%%time
dfplaces['LatLong'] = dfplaces.apply(lambda x: utmToLatLng(x['x'], x['y']), axis=1)

CPU times: user 146 ms, sys: 27.5 ms, total: 174 ms
Wall time: 192 ms


In [47]:
%%time
dfplaces[['Lat','Long']] = pd.DataFrame(dfplaces.LatLong.values.tolist(), index= dfplaces.index)

CPU times: user 3.52 ms, sys: 2.33 ms, total: 5.84 ms
Wall time: 7.4 ms


In [48]:
dfplaces=dfplaces.drop(columns=['LatLong'])

(4117, 8)

In [50]:
dfplaces.head()

,tipo_elem,id,cod_cent,nombre,x,y,Lat,Long
0,URB,3840,01001,Jose Ortega y Gasset E-O - Pº Castellana-Serrano,441615.343347,4.475768e+06,40.430502,-3.688323
1,URB,3841,01002,Jose Ortega y Gasset O-E - Serrano-Pº Castellana,441705.882340,4.475770e+06,40.430524,-3.687256
2,URB,3842,01003,Pº Recoletos N-S - Almirante-Prim,441319.371258,4.474841e+06,40.422132,-3.691727
3,URB,3843,01004,"Pº Recoletos S-N - Pl, Cibeles- Recoletos",441301.632986,4.474764e+06,40.421433,-3.691929
4,URB,3844,01005,(AFOROS) Pº Castellana S-N - Eduardo Dato - P...,441605.765072,4.476132e+06,40.433782,-3.688470


In [51]:
%%time
df_complete=df.join(dfplaces.set_index('id')[['Lat','Long']],on='id',how='left',rsuffix='_other')

CPU times: user 13.8 s, sys: 19.5 s, total: 33.3 s
Wall time: 37.7 s


In [53]:
df_complete.dtypes

id                              int64
tipo_elem                      object
intensidad                      int64
ocupacion                       int64
carga                           int64
vmed                            int64
error                          object
periodo_integracion             int64
Fecha                          object
Hora                           object
Date                   datetime64[ns]
Lat                           float64
Long                          float64
dtype: object

- Identifying columns with missing values

In [54]:
df_complete.isnull().any()

id                     False
tipo_elem              False
intensidad             False
ocupacion              False
carga                  False
vmed                   False
error                  False
periodo_integracion    False
Fecha                  False
Hora                   False
Date                   False
Lat                     True
Long                    True
dtype: bool

In [55]:
nan_rows = df_complete[df_complete['Lat'].isnull()]
nan_rows.shape

(169460, 13)

In [56]:
df_complete.shape

(57607004, 13)

169.460 Missing Values of 57.607.004 Total Values. In this first approach, we will drop them.